In [ ]:
pip install torch==1.13.1

In [ ]:
!pip uninstall torch-scatter -y
!pip uninstall torch-sparse -y
!pip uninstall pyg-lib -y
!pip uninstall git+https://github.com/pyg-team/pytorch_geometric.git -y

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [1]:
import numpy as np
import pandas as pd
import random
# from neo4j import GraphDatabase
from torch_geometric.data import Data
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from tqdm import tqdm
from collections import defaultdict
import json
import multiprocessing
import matplotlib.pyplot as plt

from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.metrics import mean_absolute_error, mean_squared_error

/home/seankhatiri/sean2/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
#### DATA LOADER ####
from torch_geometric.data import download_url, extract_zip
from torch import Tensor

def data_loader(ratings_df):
    # Create a mapping from unique user indices to range [0, num_user_nodes):
    unique_user_id = ratings_df['userId'].unique()
    unique_user_id = pd.DataFrame(data={
        'userId': unique_user_id,
        'mappedID': pd.RangeIndex(len(unique_user_id)),
    })
    print("Mapping of user IDs to consecutive values:")
    print("==========================================")
    print(unique_user_id.head())
    print()
    # Create a mapping from unique movie indices to range [0, num_movie_nodes):
    unique_item_id = ratings_df['itemId'].unique()
    unique_item_id = pd.DataFrame(data={
        'itemId': unique_item_id,
        'mappedID': pd.RangeIndex(len(unique_item_id)),
    })
    print("Mapping of item IDs to consecutive values:")
    print("===========================================")
    print(unique_item_id.head())

    ratings_user_id = pd.merge(ratings_df['userId'], unique_user_id,
                                left_on='userId', right_on='userId', how='left')
    ratings_user_id = torch.from_numpy(ratings_user_id['mappedID'].values)
    ratings_item_id = pd.merge(ratings_df['itemId'], unique_item_id,
                                left_on='itemId', right_on='itemId', how='left')
    ratings_item_id = torch.from_numpy(ratings_item_id['mappedID'].values)
    # With this, we are ready to construct our `edge_index` in COO format
    # following PyG semantics:
    edge_index_user_to_item = torch.stack([ratings_user_id, ratings_item_id], dim=0)
    # assert edge_index_user_to_item.size() == (2, 100836)
    print()
    print("Final edge indices pointing from users to items:")
    print("=================================================")
    print(edge_index_user_to_item)
    return unique_user_id, edge_index_user_to_item

# ##### MOVIELENS DATA ########
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '.'), '.')
movies_path = './ml-latest-small/movies.csv'
ratings_path = './ml-latest-small/ratings.csv'
movies_df = pd.read_csv(movies_path, index_col='movieId')
movies_df = movies_df.rename(columns={'movieId': 'itemId'})
genres = movies_df['genres'].str.get_dummies('|')
print(genres[["Action", "Adventure", "Drama", "Horror"]].head())
movie_feat = torch.from_numpy(genres.values).to(torch.float)
# assert movie_feat.size() == (9742, 20)  # 20 genres in total.
movies_ratings_df = pd.read_csv(ratings_path)
movies_ratings_df = movies_ratings_df.rename(columns={'movieId': 'itemId'})
unique_user_id, edge_index_user_to_movie = data_loader(movies_ratings_df)

###### CONTRACTS DATA ########
# contracts_ratings_df = pd.read_parquet('dataset/user_contract_rating.parquet')
# # contracts_ratings_df = contracts_ratings_df[:10000]
# contracts_df = {}
# contracts_df['name'] = contracts_ratings_df['item'].unique()
# contracts_df['itemId'], unique_names = pd.factorize(contracts_df['name'])
# contracts_df['itemId'] = contracts_df['itemId'] + 1
# contracts_df = pd.DataFrame(contracts_df, columns=['itemId', 'name'])

# contract_top_words_df = pd.read_parquet('dataset/contract_top_words.parquet')
# contract_top_words_df = contract_top_words_df.rename(columns={'contract_name': 'name'})
# contracts_df_top_words = contracts_df.merge(contract_top_words_df, on='name', how='left')
# contracts_df_top_words['keywords'] = contracts_df_top_words['keywords'].fillna('')
# contracts_df = contracts_df_top_words
# contracts_df.set_index('itemId', inplace=True)
# # f =5
# contracts_df['truncated_keywords'] = contracts_df['keywords'].apply(lambda x: ','.join(x.split(',')))
# X_df = contracts_df['truncated_keywords'].str.get_dummies(',')
# contract_feat = torch.tensor(X_df.values, dtype=torch.float)
# print(contract_feat.shape[1])

# contracts_ratings_df = contracts_ratings_df.rename(columns={'user': 'userId', 'item': 'itemId'})
# unique_user_id, edge_index_user_to_contract = data_loader(contracts_ratings_df)

Using existing file ml-latest-small.zip
Extracting ./ml-latest-small.zip


         Action  Adventure  Drama  Horror
movieId                                  
1             0          1      0       0
2             0          1      0       0
3             0          0      0       0
4             0          0      1       0
5             0          0      0       0
Mapping of user IDs to consecutive values:
   userId  mappedID
0       1         0
1       2         1
2       3         2
3       4         3
4       5         4

Mapping of item IDs to consecutive values:
   itemId  mappedID
0       1         0
1       3         1
2       6         2
3      47         3
4      50         4

Final edge indices pointing from users to items:
tensor([[   0,    0,    0,  ...,  609,  609,  609],
        [   0,    1,    2,  ..., 3121, 1392, 2873]])


In [28]:
########## PYG BINARY LINK PREDICTION ############
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

def GNN_recommender(unique_user_id, items_df, item_feat, edge_index_user_to_item):
    data = HeteroData()
    data["user"].node_id = torch.arange(len(unique_user_id))
    data["item"].node_id = torch.arange(len(items_df))
    data["item"].x = item_feat
    data["user", "rates", "item"].edge_index = edge_index_user_to_item
    data = T.ToUndirected()(data)

    # For this, we first split the set of edges into
    # training (80%), validation (10%), and testing edges (10%).
    # Across the training edges, we use 70% of edges for message passing,
    # and 30% of edges for supervision.
    # We further want to generate fixed negative edges for evaluation with a ratio of 2:1.
    # Negative edges during training will be generated on-the-fly.
    # We can leverage the `RandomLinkSplit()` transform for this from PyG:
    transform = T.RandomLinkSplit(
        num_val=0.1,
        num_test=0.1,
        disjoint_train_ratio=0.3,
        neg_sampling_ratio=2.0,
        add_negative_train_samples=False,
        edge_types=("user", "rates", "item"),
        rev_edge_types=("item", "rev_rates", "user"), 
    )
    train_data, val_data, test_data = transform(data)

    # In the first hop, we sample at most 20 neighbors.
    # In the second hop, we sample at most 10 neighbors.
    # In addition, during training, we want to sample negative edges on-the-fly with
    # a ratio of 2:1.
    # We can make use of the `loader.LinkNeighborLoader` from PyG:
    from torch_geometric.loader import LinkNeighborLoader

    # Define seed edges:
    edge_label_index = train_data["user", "rates", "item"].edge_label_index
    edge_label = train_data["user", "rates", "item"].edge_label
    train_loader = LinkNeighborLoader(
        data=train_data,
        num_neighbors=[20, 10],
        neg_sampling_ratio=2.0,
        edge_label_index=(("user", "rates", "item"), edge_label_index),
        edge_label=edge_label,
        batch_size=128,
        shuffle=True,
    )

    from torch_geometric.nn import SAGEConv, to_hetero
    import torch.nn.functional as F
    class GNN(torch.nn.Module):
        def __init__(self, hidden_channels):
            super().__init__()
            self.conv1 = SAGEConv(hidden_channels, hidden_channels)
            self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
            x = F.relu(self.conv1(x, edge_index))
            x = self.conv2(x, edge_index)
            return x
    # Our final classifier applies the dot-product between source and destination
    # node embeddings to derive edge-level predictions:
    class Classifier(torch.nn.Module):
        def forward(self, x_user: Tensor, x_item: Tensor, edge_label_index: Tensor) -> Tensor:
            edge_feat_user = x_user[edge_label_index[0]] # Convert node embeddings to edge-level representations:
            edge_feat_item = x_item[edge_label_index[1]]
            scores = (edge_feat_user * edge_feat_item).sum(dim=-1)
            return scores # Apply dot-product to get a prediction per supervision edge:

    class SimpleClassifier(torch.nn.Module):
        def forward(self, x_user: Tensor, x_item: Tensor, edge_label_index: Tensor) -> Tensor:
            edge_feat_user = x_user[edge_label_index[0]] # Convert node embeddings to edge-level representations:
            edge_feat_item = x_item[edge_label_index[1]]
            scores = torch.matmul(edge_feat_user, edge_feat_item.t())
            return torch.sigmoid(scores)
        
    class Model(torch.nn.Module):
        def __init__(self, hidden_channels):
            super().__init__()
            # Since the dataset does not come with rich features, we also learn two
            # embedding matrices for users and items:
            self.item_lin = torch.nn.Linear(20, hidden_channels) #put contract_feat.shape[1] for contracts
            self.user_emb = torch.nn.Embedding(data["user"].num_nodes, hidden_channels)
            self.item_emb = torch.nn.Embedding(data["item"].num_nodes, hidden_channels)
            # Instantiate homogeneous GNN:
            self.gnn = GNN(hidden_channels)
            # Convert GNN model into a heterogeneous variant:
            self.gnn = to_hetero(self.gnn, metadata=data.metadata())
            self.classifier = Classifier()
            # self.simple_classifier = SimpleClassifier()
        def forward(self, data: HeteroData) -> Tensor:
            x_dict = {
            "user": self.user_emb(data["user"].node_id),
            "item": self.item_lin(data["item"].x) + self.item_emb(data["item"].node_id),
            } 
            # `x_dict` holds feature matrices of all node types
            # `edge_index_dict` holds all edge indices of all edge types
            x_dict = self.gnn(x_dict, data.edge_index_dict)
            pred = self.classifier(
                x_dict["user"],
                x_dict["item"],
                data["user", "rates", "item"].edge_label_index,
            )
            return pred
            
    model = Model(hidden_channels=64)

    import tqdm
    import torch.nn.functional as F
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: '{device}'")
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(1, 20):
        total_loss = total_examples = 0
        for sampled_data in tqdm.tqdm(train_loader):
            optimizer.zero_grad()
            sampled_data.to(device)
            pred = model(sampled_data)
            ground_truth = sampled_data["user", "rates", "item"].edge_label
            loss = F.binary_cross_entropy_with_logits(pred, ground_truth)
            loss.backward()
            optimizer.step()
            total_loss += float(loss) * pred.numel()
            total_examples += pred.numel()

        # TODO: Add the val_loader, keep the best model
        print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

    # Define the validation seed edges:
    edge_label_index = val_data["user", "rates", "item"].edge_label_index
    edge_label = val_data["user", "rates", "item"].edge_label
    # val_data has neg samples in it
    val_loader = LinkNeighborLoader(
        data=val_data,
        num_neighbors=[20, 10],
        edge_label_index=(("user", "rates", "item"), edge_label_index),
        edge_label=edge_label,
        batch_size=3 * 128,
        shuffle=False,
    )
    sampled_data = next(iter(val_loader))
    from sklearn.metrics import roc_auc_score
    preds = []
    ground_truths = []
    for sampled_data in tqdm.tqdm(val_loader):
        with torch.no_grad():
            sampled_data.to(device)
            preds.append(model(sampled_data))
            ground_truths.append(sampled_data["user", "rates", "item"].edge_label)
    pred = torch.cat(preds, dim=0).cpu().numpy()
    ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
    auc = roc_auc_score(ground_truth, pred)
    print()
    print(f"Validation AUC: {auc:.4f}")
    return data, train_data, val_data, train_loader, val_loader, ground_truth, pred, test_data

In [29]:
########## EXPERIMENTS ############
###### MOVIELENS MODEL ######
data, train_data, val_data, train_loader, val_loader, ground_truth, pred, test_data = GNN_recommender(unique_user_id, movies_df, movie_feat, edge_index_user_to_movie)

###### CONTRACTS MODEL ######
# data, train_data, val_data, train_loader, val_loader, ground_truth, pred = GNN_recommender(unique_user_id, contracts_df, contract_feat, edge_index_user_to_contract)

# Ablation study of GNN (make contract_feat all zero)
# INFUTURE_ add timestamp to ?, then remove timestamp, add social_graph (user-user txs)

# ablation_without_contract_feature = torch.zeros_like(contract_feat)
# data, train_data, val_data, train_loader, val_loader, ground_truth, pred = GNN_recommender(unique_user_id, contracts_df, ablation_without_contract_feature, edge_index_user_to_contract)

# ablation_without_movie_feature = torch.zeros_like(movie_feat)
# data, train_data, val_data, train_loader, val_loader, ground_truth, pred = GNN_recommender(unique_user_id, movies_df, movie_feat, edge_index_user_to_movie)

# Cold Start User

# Cold Start Item

# Diversity

# Contract Representation (effect of f top-keywords in contract_feat)

# Sparsity 

Device: 'cuda'


100%|██████████| 190/190 [00:03<00:00, 58.86it/s]


Epoch: 001, Loss: 0.4357


100%|██████████| 190/190 [00:03<00:00, 48.31it/s]


Epoch: 002, Loss: 0.3469


...

Epoch: 018, Loss: 0.2125


100%|██████████| 190/190 [00:03<00:00, 47.99it/s]


Epoch: 019, Loss: 0.2114


100%|██████████| 79/79 [00:00<00:00, 113.92it/s]


Validation AUC: 0.9315


In [40]:
# Fucking lost the name-level train and pred
print(val_loader.data)
val_loader_df_index = val_loader.data['user', 'rates', 'item'].edge_label_index.numpy()
val_loader_df_label = val_loader.data['user', 'rates', 'item'].edge_label.numpy()
val_loader_df_index = val_loader_df_index.T 
val_loader_df = pd.DataFrame(val_loader_df_index, columns=['user', 'item'])
val_loader_df['rating'] = val_loader_df_label

train_loader_df_index = train_loader.data['user', 'rates', 'item'].edge_label_index.numpy()
train_loader_df_label = train_loader.data['user', 'rates', 'item'].edge_label.numpy()
train_loader_df_index = train_loader_df_index.T 
train_loader_df = pd.DataFrame(train_loader_df_index, columns=['user', 'item'])
train_loader_df['rating'] = train_loader_df_label

from lightfm import LightFM
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit(data['user'].node_id.numpy(), data['item'].node_id.numpy())
user_ids_mapping, _, item_ids_mapping, _ = dataset.mapping()

(train_interactions, train_interactions_weight) = dataset.build_interactions((row['user'], row['item'], row['rating']) for index, row in train_loader_df.iterrows())

model = LightFM(loss='warp')
model.fit(train_interactions, epochs=30, num_threads=2, sample_weight=train_interactions_weight)

val_loader_df['pred'] = 0
for i, row in val_loader_df.iterrows():
    user_id_internal = user_ids_mapping[row['user']]
    item_id_internal = item_ids_mapping[row['item']]
    val_loader_df['pred'][i] = model.predict(user_id_internal, [item_id_internal])

pred_mf = val_loader_df['pred'].to_numpy()
ground_truth_mf = val_loader_df['rating'].to_numpy()

HeteroData(
  user={ node_id=[610] },
  item={
    node_id=[9742],
    x=[9742, 20],
  },
  (user, rates, item)={
    edge_index=[2, 80670],
    edge_label=[30249],
    edge_label_index=[2, 30249],
  },
  (item, rev_rates, user)={ edge_index=[2, 80670] },
  (user, rats, item)={},
  (user, ratse, item)={}
)


/tmp/ipykernel_4060785/3375834210.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_loader_df['pred'][i] = model.predict(user_id_internal, [item_id_internal])


In [41]:
# in val_data len(edge_index) = 80670, but len(edge_label_index) = 30249, we selected edge_label_index since for train_loader used the same

def precision_at_k(user_id, edge_index, ground_truth, pred, k):

    mask = edge_index[0] == user_id
    filtered_pred = pred[mask]
    filtered_ground_truth = ground_truth[mask]
    
    sorted_indices = filtered_pred.argsort()[:: -1]
    top_k = [(filtered_ground_truth[i], filtered_pred[i]) for i in sorted_indices[:k]]
    hit = 0
    for i in range(len(top_k)):
        ground_truth, pred = top_k[i]
        if ground_truth > 0 and pred > 0: # I think we should remove this: and pred > 0:
            hit += 1
    precision = hit / k

    return precision


def ap_at_k(k, precision_at_k, mode):
    precisions = []
    edge_index = val_loader.data['user', 'rates', 'item'].edge_label_index
    for user_id in tqdm(edge_index[0], total=len(edge_index[0])):
        if mode == 'mf':
            precisions.append(precision_at_k(user_id, edge_index, ground_truth, pred_mf, k)) # ground_truth is the same for both GNN and mf
        else:
            precisions.append(precision_at_k(user_id, edge_index, ground_truth, pred, k))

    return np.mean(precisions)


k_values = [1, 2, 3, 4, 5]
for k in k_values:
    hit_at_k = ap_at_k(k, precision_at_k, mode='mf')
    print(f"AP@{k}:", hit_at_k)


  0%|          | 0/30249 [00:00<?, ?it/s]

100%|██████████| 30249/30249 [00:05<00:00, 5304.64it/s]


AP@1: 0.3861615260008595


100%|██████████| 30249/30249 [00:05<00:00, 5208.43it/s]


AP@2: 0.3345399847928857


...

AP@3: 0.2895412520524095


100%|██████████| 30249/30249 [00:06<00:00, 5014.00it/s]


AP@4: 0.24752884392872493


100%|██████████| 30249/30249 [00:06<00:00, 4697.94it/s]

AP@5: 0.21366656748983437


In [19]:
print(len(set(val_data['user', 'rates', 'item'].edge_label_index.numpy()[0])))
print(len(set(val_data['user', 'rates', 'item'].edge_label_index.numpy()[1])))
print(len(movies_df['title'].unique()))

610
9006
9737
